<a href="https://www.kaggle.com/code/hassanabsar/amazon-deals-today-s-deals-data-cleaning?scriptVersionId=176685236" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Amazon | Deals (Today's Deals) - Data Cleaning

In [1]:
# importing liabraries
import pandas as pd

In [2]:
#  importing and disabling warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# setting options to show maximum of row and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
# importing dataset into dataframe
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        files = os.path.join(dirname, filename)
        print(files)

/kaggle/input/amazon-deals-todays-deals/raw_Data_Amazon _ Deals.csv
/kaggle/input/amazon-deals-todays-deals/clean_data_amazon_deals.csv


In [5]:
# importing datasets
df=pd.read_csv('/kaggle/input/amazon-deals-todays-deals/raw_Data_Amazon _ Deals.csv')

In [6]:
df.shape

(23543, 21)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23543 entries, 0 to 23542
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Image             23543 non-null  object
 1   alinknormal_URL   23543 non-null  object
 2   Label             23543 non-null  object
 3   Time              23503 non-null  object
 4   alinknormal_URL1  23543 non-null  object
 5   Content           23543 non-null  object
 6   Title             22959 non-null  object
 7   Title_URL         22959 non-null  object
 8   Image1            20864 non-null  object
 9   asizebase         20864 non-null  object
 10  aiconalt          19113 non-null  object
 11  asizesmall        19114 non-null  object
 12  Label1            22959 non-null  object
 13  Price             4900 non-null   object
 14  aoffscreen        22959 non-null  object
 15  Price1            22959 non-null  object
 16  Keywords          4900 non-null   object
 17  asizemini   

In [8]:
# fining missing values in the df
df.isnull().sum()

Image                   0
alinknormal_URL         0
Label                   0
Time                   40
alinknormal_URL1        0
Content                 0
Title                 584
Title_URL             584
Image1               2679
asizebase            2679
aiconalt             4430
asizesmall           4429
Label1                584
Price               18643
aoffscreen            584
Price1                584
Keywords            18643
asizemini             584
asizemini2            635
asizemini_URL         635
asizemini3            635
dtype: int64

In [9]:
# Renaming some of the columns for better understanding because scrapping has been done on element_names
df = df.rename(columns={'Image': 'image_url', 'alinknormal_URL':'product_url','Label':'price_discount_group', 
                        'Time':'deal_time_limit', 'Content':'product_type', 'Title':'product_name','Title_URL':'brand_url', 
                        'asizebase':'brand_name','aiconalt':'rating_outof5', 'asizesmall':'rated_by', 'Price':'discount',
                        'aoffscreen':'disc_price', 'Keywords':'list_price', 'asizemini2':'log_type' })

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23543 entries, 0 to 23542
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   image_url             23543 non-null  object
 1   product_url           23543 non-null  object
 2   price_discount_group  23543 non-null  object
 3   deal_time_limit       23503 non-null  object
 4   alinknormal_URL1      23543 non-null  object
 5   product_type          23543 non-null  object
 6   product_name          22959 non-null  object
 7   brand_url             22959 non-null  object
 8   Image1                20864 non-null  object
 9   brand_name            20864 non-null  object
 10  rating_outof5         19113 non-null  object
 11  rated_by              19114 non-null  object
 12  Label1                22959 non-null  object
 13  discount              4900 non-null   object
 14  disc_price            22959 non-null  object
 15  Price1                22959 non-null

In [11]:
# splitting the columns to get some meaningful insights
df.insert(18, "listed_price", df["list_price"].str.split(":").str[1])
df['ratings']=df["rating_outof5"].str.split(" ").str[0]

In [12]:
# Selecting only the necessary and meaningful columns as they are required for our analysis
df = df.loc[:, ['image_url', 'product_url', 'price_discount_group', 'product_type', 'product_name', 'brand_url', 
                'brand_name', 'ratings', 'rated_by', 'discount', 'disc_price', 'listed_price', 'log_type']]

In [13]:
# removing prefixing and suffixing white spaces from all columns
df = df.apply(lambda x: x.str.strip())

In [14]:
# Handling replacements in string columns
df['product_type'] = df['product_type'].str.replace("Save on ", "", case=False, regex=False)
df['rated_by'] = df['rated_by'].str.replace(",", "", case=False, regex=False)
df['disc_price'] = df['disc_price'].str.replace(",", "", case=False, regex=False)
df['listed_price'] = df['listed_price'].str.replace(",", "", case=False, regex=False)
df['discount'] = df['discount'].str.replace("-", "", case=False, regex=False)
df['disc_price'] = df['disc_price'].str.replace("AED ", "", case=False, regex=False)
df['listed_price'] = df['listed_price'].str.replace("AED ", "", case=False, regex=False)
df['rated_by'] = df['rated_by'].str.replace("(AED 0.13/gram)", "0", case=False, regex=False)

In [15]:
# Change column type to float64 for columns: 'ratings', 'rated_by', 'disc_price', and 'listed_price'
df = df.astype({'ratings': 'float64','disc_price': 'float64' , 'listed_price': 'float64', 'rated_by': 'float64'})

In [16]:
#drop the rows where 'product_name' is null
df.dropna(subset=['product_name'], inplace=True)
#filling the null values in 'listed_price' column with the corresponding values in 'disc_price' column
df["listed_price"].fillna(df["disc_price"], inplace=True)
# replacing the nulls in 'brand_name' column with unknown
df["brand_name"].fillna("Unknown", inplace=True)
# replace the missing values in 'log_type' with 'PAID Freight'
df['log_type'].fillna("PAID Freight", inplace=True)
# fill the missing values of 'ratings' column with average of corresponding 'product_type' column
df["ratings"].fillna(df.groupby("product_type")["ratings"].transform("mean"), inplace=True)
# replacing the missing values in 'rated_by' column with 0
df = df.fillna({'rated_by': 0})

In [17]:
# filling the null values in 'discount' column with 0%
df["discount"].fillna("0%", inplace=True)
#checking if there are any rows where 'discount' is 0% but 'listed_price' is different than 'disc_price'
df_0_disc = df[(df["discount"] == "0%") & (df["listed_price"] != df["disc_price"])]
#checking if there are any rows where 'discount' is not equal to 0% but 'listed_price' is same as 'disc_price'
df_not_0_disc = df[(df["discount"] != "0%") & (df["listed_price"] == df["disc_price"])]
print(len(df_not_0_disc), len(df_0_disc))

0 0


In [18]:
#finding duplicates
df[df.duplicated()].count()

image_url               597
product_url             597
price_discount_group    597
product_type            597
product_name            597
brand_url               597
brand_name              597
ratings                 597
rated_by                597
discount                597
disc_price              597
listed_price            597
log_type                597
dtype: int64

In [19]:
#dropping duplicates
df.drop_duplicates(inplace=True)

In [20]:
df.isnull().sum()

image_url               0
product_url             0
price_discount_group    0
product_type            0
product_name            0
brand_url               0
brand_name              0
ratings                 0
rated_by                0
discount                0
disc_price              0
listed_price            0
log_type                0
dtype: int64

In [21]:
df.shape

(22362, 13)

In [22]:
df.size

290706

In [23]:
df.head()

,image_url,product_url,price_discount_group,product_type,product_name,brand_url,brand_name,ratings,rated_by,discount,disc_price,listed_price,log_type
0,https://m.media-amazon.com/images/I/618nf5t-s3...,https://www.amazon.ae/deal/9f7030f6&showVariat...,Up to 33% off,UGREEN Electronic Accessories,HDMI Cable 4K 2M HDMI 2.0 18Gbps High-Speed 4K...,https://www.amazon.ae/UGREEN-High-Speed-Compat...,UGREEN,4.6,17914.0,25%,35.25,47.0,FREE Delivery
1,https://m.media-amazon.com/images/I/618nf5t-s3...,https://www.amazon.ae/deal/9f7030f6&showVariat...,Up to 33% off,UGREEN Electronic Accessories,"XLR Cable, Microphone XLR Male to Female Exten...",https://www.amazon.ae/UGREEN-Microphone-Extens...,UGREEN,4.6,12736.0,20%,39.20,49.0,FREE Delivery
2,https://m.media-amazon.com/images/I/618nf5t-s3...,https://www.amazon.ae/deal/9f7030f6&showVariat...,Up to 33% off,UGREEN Electronic Accessories,"Aluminum Vertical Laptop Stand, Double Desktop...",https://www.amazon.ae/UGREEN-Aluminum-Vertical...,UGREEN,4.7,2760.0,20%,71.20,89.0,FREE Delivery
3,https://m.media-amazon.com/images/I/618nf5t-s3...,https://www.amazon.ae/deal/9f7030f6&showVariat...,Up to 33% off,UGREEN Electronic Accessories,iPhone Magsafe Charger Car Wireless Charging H...,https://www.amazon.ae/UGREEN-Magsafe-Wireless-...,UGREEN,4.3,1183.0,36%,89.00,139.0,FREE Delivery
4,https://m.media-amazon.com/images/I/618nf5t-s3...,https://www.amazon.ae/deal/9f7030f6&showVariat...,Up to 33% off,UGREEN Electronic Accessories,Mini USB Cable USB 2.0 Type A to Mini B Cable ...,https://www.amazon.ae/UGREEN-Compatible-Contro...,UGREEN,4.6,29832.0,20%,26.40,33.0,FREE Delivery


In [24]:
df.to_csv('amazon_today_deals.csv')  # saving the clean data to a csv file